In [3]:
# import cell
%matplotlib inline
import glob
import cv2
import numpy as np
import scipy.misc as smp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [8]:
def read_image(path):
    images = []
    for img in glob.glob(path):
        read_image = cv2.imread(img)
#         read_image = read_image.astype(np.float32)
        images.append(read_image)
        
    return images

In [ ]:
images = read_image('data/IMG/*')
plt.imshow(images[0])